# LangChain Chat Part 2 - Conversation

From DeepLearning.ai's `LangChain Chat with Your Data` course.

Tutorial: https://learn.deeplearning.ai/langchain-chat-with-your-data/lesson/7/chat

Be sure to persist vector store to disk by running langchain_chat_1_persist_vectore_store notebook before running this one.

## Load Vectore Store from Filesystem

In [1]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

persist_directory = 'chroma/'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

## Enable Debug

In [2]:
from langchain.globals import set_debug, set_verbose

# set_debug overrides set_verbose
set_debug(True)
# set_verbose(True)

## Build Memory

In [3]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
retriever=vectordb.as_retriever()

# ConversationRetrievalChain adds a new step on top of the RetrievalQA chain.
# It takes the history and the new question, and condenses it into a standalone question to pass to the vector store to look up relevant documents.
# It also uses this condensed standalone question to pass to the language model to get an answer along with the relevant chunks from the vector store.
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory,
    # verbose=True # verbose doesn't seem to do anything anymore now that we set_debug(True)
)

In [5]:
question = "Is probability a class topic?"
result = qa({"question": question})
print(result["answer"])

[chain/start] [1:chain:ConversationalRetrievalChain] Entering Chain run with input:
{
  "question": "Is probability a class topic?",
  "chat_history": []
}
[chain/start] [1:chain:ConversationalRetrievalChain > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:ConversationalRetrievalChain > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "Is probability a class topic?",
  "context": "of this class will not be very program ming intensive, although we will do some \nprogramming, mostly in either MATLAB or Octa ve. I'll say a bit more about that later.  \nI also assume familiarity with basic proba bility and statistics. So most undergraduate \nstatistics class, like Stat 116 taught here at Stanford, will be more than enough. I'm gonna \nassume all of you know what ra ndom variables are, that all of you know what expectation \nis, what a variance or a random variable is. And in case of some of you, it's

In [6]:
question = "why are those prerequesites needed?"
result = qa({"question": question})
print(result['answer'])

[chain/start] [1:chain:ConversationalRetrievalChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:ConversationalRetrievalChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "question": "why are those prerequesites needed?",
  "chat_history": "\nHuman: Is probability a class topic?\nAssistant: Yes, probability is a topic that will be covered in the class. The instructor assumes familiarity with basic probability and statistics, so it is expected that students have prior knowledge in this area."
}
[llm/start] [1:chain:ConversationalRetrievalChain > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n\nHuman: Is probability a class topic?\nAssistant: Yes, probability is a topic that will be covered in the class. The instructor assumes familiarity with basic p